In [ ]:
!pip install nltk
!pip install -q wordcloud
!pip install matplotlib
!pip install vaderSentiment
!pip install NRCLex
!pip install seaborn
!pip install tensorflow
!pip install keras
!pip install scikit-learn

# Import the Dataset

In [ ]:
from pandas import DataFrame as df
import matplotlib.pyplot as mp
import pandas as pd
import seaborn as sb
import matplotlib as plt

In [ ]:

from nrclex import NRCLex
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

df['fear'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('fear', 0))

df['anger'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('anger', 0))

df['anticipation'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('anticipation', 0))

df['trust'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('trust', 0))

df['surprise'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('surprise', 0))

df['positive'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('positive', 0))

df['negative'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('negative', 0))

df['sadness'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('sadness', 0))

df['disgust'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('disgust', 0))

df['joy'] = df['text'].apply(lambda x: NRCLex(x).raw_emotion_scores.get('joy', 0))

for index, row in df.iterrows():
    sentence = row['text']
    sentiment = analyzer.polarity_scores(sentence)

    df.at[index, 'negative'] = sentiment['neg']
    df.at[index, 'neutral'] = sentiment['neu']
    df.at[index, 'positive'] = sentiment['pos']
    df.at[index, 'compound'] = sentiment['compound']



In [ ]:
df['Message Size'] = df['text'].apply(lambda x: len(x))

In [ ]:
df.head(10)

In [ ]:
df1 = df.copy();

df1.drop(columns=['text'], inplace=True)

plt.figure(figsize=(15, 15))
# plotting correlation heatmap 
dataplot = sb.heatmap(df1.corr(), cmap="YlGnBu", annot=True)

# displaying heatmap 
mp.show()

In [225]:

from keras.src.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras.src.utils import pad_sequences
from keras.models import Sequential
from keras.src.layers import Dense, Embedding
import pandas as pd
from nltk.corpus import stopwords

stop = stopwords.words('english')

df = pd.read_csv("./datasets/train2Data.csv", encoding="utf-8", sep=',')

df.rename(columns={'Tweets': 'text', ' Label': 'label'}, inplace=True)

#Remove stopwords from the dataset
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['text'] = df['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in ['amp', 'quot']]))


max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)


embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dense(16))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

X_train, X_test, Y_train, Y_test = train_test_split(X,df['label'], test_size = 0.20, random_state = 42)

Model: "sequential_31"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_43 (Embedding)    (None, 36, 128)           256000    
                                                                 
 dense_57 (Dense)            (None, 36, 16)            2064      
                                                                 
 dense_58 (Dense)            (None, 36, 1)             17        
                                                                 
Total params: 258081 (1008.13 KB)
Trainable params: 258081 (1008.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [226]:
model.fit(X_train, Y_train, epochs = 7, validation_data=(X_test, Y_test),batch_size=32, verbose = 1)

Epoch 1/7
775/775 [==============================] - 4s 5ms/step - loss: 0.6424 - accuracy: 0.5896 - val_loss: 0.6376 - val_accuracy: 0.5938
Epoch 2/7
775/775 [==============================] - 3s 4ms/step - loss: 0.6370 - accuracy: 0.5947 - val_loss: 0.6373 - val_accuracy: 0.5938
Epoch 3/7
775/775 [==============================] - 3s 4ms/step - loss: 0.6366 - accuracy: 0.5962 - val_loss: 0.6373 - val_accuracy: 0.5940
Epoch 4/7
775/775 [==============================] - 3s 4ms/step - loss: 0.6360 - accuracy: 0.5975 - val_loss: 0.6372 - val_accuracy: 0.5939
Epoch 5/7
775/775 [==============================] - 3s 4ms/step - loss: 0.6359 - accuracy: 0.5977 - val_loss: 0.6371 - val_accuracy: 0.5941
Epoch 6/7
775/775 [==============================] - 3s 4ms/step - loss: 0.6357 - accuracy: 0.5975 - val_loss: 0.6368 - val_accuracy: 0.5945
Epoch 7/7
775/775 [==============================] - 3s 4ms/step - loss: 0.6357 - accuracy: 0.5978 - val_loss: 0.6382 - val_accuracy: 0.5940
